In [ ]:
import torch
import os
from IPython.display import Image, clear_output  # to display images
import shutil

# ----- YOLOv5 Setup -----
if not os.path.exists("yolov5"):
    !git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt # install
%pip install -q roboflow
# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

# ----- Dataset Configuration -----
# DATASET_CHOICE = "grozi"
# set up environment
os.environ["DATASET_DIRECTORY"] = "/content/datasets"
# ----- Roboflow Dataset Loading -----
from roboflow import Roboflow

if not REAL_IMAGE_MODE:
    if DATASET_CHOICE == "webmarket":
        dataset_path = "/content/datasets/Webmarket-4"
        model_weights = "/content/drive/MyDrive/Object_Recognition/yolo_model/webmarket/best.pt"
        if not os.path.exists(dataset_path):
            rf = Roboflow(api_key="ILw5ia3CDLaVSgIHRYon")
            project = rf.workspace("final-project-mwyx2").project("webmarket-7inwv-e4bky")
            version = project.version(4)
            dataset = version.download("yolov5")
    elif DATASET_CHOICE == "grozi":
        dataset_path = "/content/datasets/grozi-3.2k-testing-5"
        model_weights = "/content/drive/MyDrive/Object_Recognition/yolo_model/grocery_product/best.pt"
        if not os.path.exists(dataset_path):
            rf = Roboflow(api_key="ILw5ia3CDLaVSgIHRYon")
            project = rf.workspace("final-project-mwyx2").project("grozi-3.2k-testing-fzu9s")
            version = project.version(5)
            dataset = version.download("yolov5")
    else:
        raise ValueError("Unsupported dataset choice.")
    print("Dataset path:", dataset_path)
else:
    model_weights = "/content/drive/MyDrive/Object_Recognition/yolo_model/grocery_product/best.pt"
    dataset_path = "/content/datasets/Dutch-supermarket-grocery-1"
    if not os.path.exists(dataset_path):
      rf = Roboflow(api_key="ILw5ia3CDLaVSgIHRYon")
      project = rf.workspace("final-project-mwyx2").project("dutch-supermarket-grocery")
      version = project.version(1)
      dataset = version.download("yolov5")
    print("Using your own images at:", dataset_path)

# # ----- Run Detection -----
crops_dir = "runs/detect/exp/crops/0"
source_path = os.path.join(dataset_path, "test/images")

if not os.path.exists(crops_dir):
    !python detect.py --weights {model_weights} --img 640 --conf 0.5 --source "{source_path}" --save-crop --save-txt
else:
    print("Detection already exists, skipping...")

# ----- Organize Crops by Original Image -----
# Define your cropped images directory
output_dir = "organized_crops"  # Directory to store organized cropped images
# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)
# Iterate through all cropped images
for filename in os.listdir(crops_dir):
    if filename.endswith(".jpg"):
        # Extract the original image prefix (before '_')
        prefix = filename.split('_')[0]

        # Create a subdirectory for each original image if not exists
        target_dir = os.path.join(output_dir, prefix)
        os.makedirs(target_dir, exist_ok=True)

        # Move the file to its corresponding folder
        src_path = os.path.join(crops_dir, filename)
        dst_path = os.path.join(target_dir, filename)
        shutil.move(src_path, dst_path)

print("Cropped images organized successfully!")